In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import glob

In [4]:

origin_img = cv2.imread('/home/office5/Image_Code/Ketan_sir/data/Anna2.bmp',0)
thresh = cv2.threshold(origin_img,45,255,cv2.THRESH_BINARY)[1]

CONFIG_PATH = "/home/office5/Image_Code/Ketan_sir"

def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")

edge1 = origin_img[2744:3136,1824:2736]


l,x,y = [],[],[]
right_angle_point = []

for img1 in sorted(glob.glob('/home/office5/Image_Code/Ketan_sir/try_connected_component/*.png'), key=len):
    if img1 != '/home/office5/Image_Code/Ketan_sir/try_connected_component/0.png':
        img_name = img1.split('/')[-1]
        img = cv2.imread(img1)
        # print(img,img.shape)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        coords = np.column_stack(np.where(gray == 255))

        a = np.min(coords,axis=0)
        b = np.max(coords,axis=0)

        y_min,x_min,y_max,x_max = a[0],a[1],b[0],b[1]


        # For getting right angle point we are de
        gray = np.float32(gray)
        dst = cv2.cornerHarris(gray,2,3,0.04)
        ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
        dst = np.uint8(dst)

        ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
        corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

        for i in range(len(corners)):
            for j in range(1,config['measure_pixel']):
                hei_inc = int(np.int0(corners[i])[1]) + j
                wid_inc = int(np.int0(corners[i])[0]) + j
                hei_dec = int(np.int0(corners[i])[1]) - j
                wid_dec = int(np.int0(corners[i])[0]) - j
                same_hei = int(np.int0(corners[i])[1])
                same_wid = int(np.int0(corners[i])[0])

                check_extreme_point = []
                try:
                    if(gray[same_hei][same_wid]!=255):
                        break
                    else:
                        if( (gray[hei_inc][same_wid]!=255 or gray[same_hei][wid_inc]!=255) and (gray[hei_dec][same_wid]!=255 or gray[same_hei][wid_dec]!=255) and (gray[hei_dec][same_wid]!=255 or gray[same_hei][wid_inc]!=255) and (gray[hei_inc][same_wid]!=255 or gray[same_hei][wid_dec]!=255)):
                            break
                        else:
                            check_extreme_point.append(1)        
                except:
                    print("out of image")

            if(len(check_extreme_point)==1):

                # print("#-------------------------New Points -------------------------------#")

                edge1[same_hei,same_wid] = 255
                right_angle_point.append([same_hei,same_wid])

        # print(right_angle_point)

        points = []
        
        for i in range(len(coords)):
            if(x_min == coords[i][1]):
                points.append([coords[i][0], coords[i][1]]) 
                break
            else:
                pass
        points.append([coords[-1][0] , coords[-1][1]])
        points = np.array(points)
        corner_point = points.copy()
        points[0] = points[0] + np.array([2744,1824]) 
        points[1] = points[1] + np.array([2744,1824]) 

        for i in range(3):
            try:
                upp_down_a = cv2.absdiff(np.array(origin_img[points[i][0]-1][points[i][1]]) , np.array(origin_img[points[i][0]+1][points[i][1]]))
                left_right_b = cv2.absdiff(np.array(origin_img[points[i][0]][points[i][1]-1]),np.array(origin_img[points[i][0]][points[i][1]+1]))
                      
                if(upp_down_a>=30 and left_right_b>=20):
                    edge1[corner_point[i][0],corner_point[i][1]] = 255
                    origin_img[points[i][0],points[i][1]] = 255
                    right_angle_point.append([corner_point[i][0],corner_point[i][1]])

            except:
                pass

print(right_angle_point)
cv2.imwrite('/home/office5/Image_Code/Ketan_sir/extremen_point_right_angle/extreme_point2.png',edge1)
# cv2.imwrite('extreme_point_original2.png',origin_img)



[[112, 454], [51, 526], [51, 526], [31, 459], [4, 592], [33, 28], [33, 28], [113, 453], [9, 534], [88, 248], [123, 21], [56, 395], [45, 417], [46, 415], [290, 763], [292, 478], [333, 488], [344, 549], [294, 195], [207, 716], [212, 348], [377, 496], [211, 274], [344, 549], [344, 549], [233, 738], [233, 738], [285, 495], [71, 247], [262, 735], [262, 735], [262, 728], [291, 747], [286, 413], [375, 534], [292, 361], [286, 381], [286, 413], [376, 465], [262, 612], [265, 671], [262, 612], [265, 671], [292, 453], [209, 301], [294, 252], [286, 191], [294, 215], [308, 555], [255, 353], [269, 358], [287, 688], [286, 380], [378, 388], [223, 399], [243, 408], [307, 554], [336, 570], [292, 764], [291, 548], [292, 565], [295, 197], [285, 544], [308, 533], [310, 554], [375, 574], [232, 667], [246, 695], [293, 480], [370, 531], [369, 509], [205, 596], [292, 529], [370, 509], [370, 509], [274, 586], [376, 499], [376, 512], [332, 436], [309, 553], [375, 573], [208, 345], [287, 212], [231, 735], [231, 72

True

In [18]:
# np.save('coordinate.npy',right_angle_point)
# temp

In [6]:
for img1 in sorted(glob.glob('/home/office5/Image_Code/Ketan_sir/try_connected_component/*.png'), key=len):
    if img1 != '/home/office5/Image_Code/Ketan_sir/try_connected_component/0.png':
        img = cv2.imread(img1)
        print(img.shape)
        print(img)

(392, 912, 3)
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
(392, 912, 3)
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
(392, 912, 3)
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0

In [17]:
a = np.array([1,1,0,5,1,1,1])
b = np.unique(a,return_counts=True)
if (len(b[0]))==2:
    print("True")